## Set up

In [11]:
import os
os.getcwd()
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3" # 用來解決python環境版本問題

In [12]:
from pprint import pprint

import findspark # 用來解決python跑spark的時候找不到路徑問題
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

In [13]:
sc.stop()

In [14]:
conf = SparkConf().setMaster("local").setAppName("hw1")
sc = SparkContext(conf=conf)
sc # 可以叫出Spark UI來確認運行狀況

<SparkContext master=local appName=hw1>

## Input

將檔案讀入`lines`變數，等待後續處理

In [39]:
lines = sc.textFile("inputdata.txt")

## Preprocess

### split and map into RDD

定義`mapper_input()`function，用來將原本每一行的資料切分和整理  
    輸入格式： `M,0,0,10`  
    輸出格式： `('M', ['0', '0', '10']),`
    

In [40]:
def mapper_input(line):
    matrix_list = line.split(",")
    matrix_map = (matrix_list[0], [matrix_list[1],matrix_list[2],matrix_list[3]])
    return matrix_map

In [41]:
linesRDD = lines.map(mapper_input)

### get martix M and N by filter

因為有了`'M'`或是`'N'`的key值，所以可以將資料分成M矩陣和N矩陣  
這邊用RDD的`filter`來進行

In [42]:
M_RDD = linesRDD.filter(lambda x : "M" in x[0]) # [0]is the key value

In [43]:
N_RDD = linesRDD.filter(lambda x : "N" in x[0])

###  Multiply

$$
M_{ij}=\begin{bmatrix}
    m_{11}&m_{12}&...&m_{1j}\\\\
    m_{21}&m_{22}&...&m_{2j}\\\\
    &...\\\\
    m_{i1}&m_{i2}&...&m_{ij}
    \end{bmatrix}
    =\begin{bmatrix}
    m'_{1}\\\\
    m'_{2}\\\\
    ...\\\\
    m'_{i}
    \end{bmatrix}
,
N_{jk}=\begin{bmatrix}
    n_{11}&n_{12}&...&n_{1k}\\\\
    n_{21}&n_{22}&...&n_{2k}\\\\
    &...\\\\
    n_{j1}&n_{j2}&...&n_{jk}
    \end{bmatrix}
    =\begin{bmatrix}
    n'_{1}&n'_{2}&...&n'_{k}
    \end{bmatrix}
$$
因為$P_{ik}=M_{ij}N_{jk}$,兩個矩陣可以相乘的話一定要有共同的j，所以將M和N分別以row vector和column vector來表示以便進行後續的計算  
**註：25、27行是Example時輸出的結果，作為格式參考,實際操作可以先不用運行（等到後續一次跑完再跑collect就好）**

In [44]:
M_RDD_matrix = M_RDD.map(lambda x : (x[1][1], [x[0],x[1][0],x[1][2]] ))

In [25]:
# (j,[M,i,m_ij])
M_RDD_matrix.collect() 

[('0', ['M', '0', '10']),
 ('1', ['M', '0', '0']),
 ('2', ['M', '0', '20']),
 ('0', ['M', '1', '0']),
 ('1', ['M', '1', '30']),
 ('2', ['M', '1', '0']),
 ('0', ['M', '2', '40']),
 ('1', ['M', '2', '0']),
 ('2', ['M', '2', '50'])]

In [45]:
N_RDD_matrix = N_RDD.map(lambda x : (x[1][0], [x[0],x[1][1],x[1][2]] ))

In [27]:
# (j,[N,k,n_jk])
N_RDD_matrix.collect()

[('0', ['N', '0', '1']),
 ('0', ['N', '1', '2']),
 ('0', ['N', '2', '3']),
 ('1', ['N', '0', '4']),
 ('1', ['N', '1', '5']),
 ('1', ['N', '2', '0']),
 ('2', ['N', '0', '6']),
 ('2', ['N', '1', '7']),
 ('2', ['N', '2', '8'])]

> 在進行大矩陣運算的時候的重點在於找出可以不會互相影響的步驟而拆開進行

這邊我們注意到在$m'_{\alpha}$和$n'_{\beta}$相乘的時候是可以互相獨立的，所以我們用`join()`的方式讓它們兩兩相乘  
所以就會得到 $key=j,  value=([M,i,M_{ij}],[N,k,N{jk}]) $的RDD  
**註：29行是Example時輸出的結果，作為格式參考,實際操作可以先不用運行（等到後續一次跑完再跑collect就好）**

In [46]:
MN_RDD = M_RDD_matrix.join(N_RDD_matrix)

In [29]:
MN_RDD.collect()

[('0', (['M', '0', '10'], ['N', '0', '1'])),
 ('0', (['M', '0', '10'], ['N', '1', '2'])),
 ('0', (['M', '0', '10'], ['N', '2', '3'])),
 ('0', (['M', '1', '0'], ['N', '0', '1'])),
 ('0', (['M', '1', '0'], ['N', '1', '2'])),
 ('0', (['M', '1', '0'], ['N', '2', '3'])),
 ('0', (['M', '2', '40'], ['N', '0', '1'])),
 ('0', (['M', '2', '40'], ['N', '1', '2'])),
 ('0', (['M', '2', '40'], ['N', '2', '3'])),
 ('1', (['M', '0', '0'], ['N', '0', '4'])),
 ('1', (['M', '0', '0'], ['N', '1', '5'])),
 ('1', (['M', '0', '0'], ['N', '2', '0'])),
 ('1', (['M', '1', '30'], ['N', '0', '4'])),
 ('1', (['M', '1', '30'], ['N', '1', '5'])),
 ('1', (['M', '1', '30'], ['N', '2', '0'])),
 ('1', (['M', '2', '0'], ['N', '0', '4'])),
 ('1', (['M', '2', '0'], ['N', '1', '5'])),
 ('1', (['M', '2', '0'], ['N', '2', '0'])),
 ('2', (['M', '0', '20'], ['N', '0', '6'])),
 ('2', (['M', '0', '20'], ['N', '1', '7'])),
 ('2', (['M', '0', '20'], ['N', '2', '8'])),
 ('2', (['M', '1', '0'], ['N', '0', '6'])),
 ('2', (['M', '1', '

因為已經將元素乘完了，所以$j,M,N$都已經沒有用了，利用map將它們去除  
留下有用的$key=(i,k), value=m_{ij}\times n_{jk}$  
**註：31行是Example時輸出的結果，作為格式參考,實際操作可以先不用運行（等到後續一次跑完在跑collect就好）**

In [47]:
P_RDD_matrix = MN_RDD.map(lambda x: ((int(x[1][0][1]),int(x[1][1][1])),int(x[1][0][2])*int(x[1][1][2])) )

In [31]:
P_RDD_matrix.collect()

[((0, 0), 10),
 ((0, 1), 20),
 ((0, 2), 30),
 ((1, 0), 0),
 ((1, 1), 0),
 ((1, 2), 0),
 ((2, 0), 40),
 ((2, 1), 80),
 ((2, 2), 120),
 ((0, 0), 0),
 ((0, 1), 0),
 ((0, 2), 0),
 ((1, 0), 120),
 ((1, 1), 150),
 ((1, 2), 0),
 ((2, 0), 0),
 ((2, 1), 0),
 ((2, 2), 0),
 ((0, 0), 120),
 ((0, 1), 140),
 ((0, 2), 160),
 ((1, 0), 0),
 ((1, 1), 0),
 ((1, 2), 0),
 ((2, 0), 300),
 ((2, 1), 350),
 ((2, 2), 400)]

因為對於每一個$P_{\alpha\beta}$來說，都會有$j$個來源  
我們前面只做了將它們乘起來，這邊再用`reduceByKey()`來將同一個位置的$P_{\alpha\beta}$加在一起

In [48]:
P_RDD = P_RDD_matrix.reduceByKey(lambda x,y: int(x)+int(y))

### Output

為了符合輸出個格式，先用`sortByKey()`來排序

In [53]:
P_RDD_sort = P_RDD.sortByKey()

前面的`collect()`都可以先不用跑，等到全部計算步驟做完之後再跑一次就可以了  
因為對於spark來說前面在進行的都是「**轉換**」運算，不會馬上執行  
而`collect()`是一個「**動作**」運算，所以會立刻執行

In [54]:
P_RDD_sort.collect()

[((0, 0), 1224217),
 ((0, 1), 1209205),
 ((0, 2), 1172528),
 ((0, 3), 1150746),
 ((0, 4), 1152989),
 ((0, 5), 1143435),
 ((0, 6), 1192906),
 ((0, 7), 1163613),
 ((0, 8), 1146928),
 ((0, 9), 1270345),
 ((0, 10), 1164367),
 ((0, 11), 1197043),
 ((0, 12), 1197989),
 ((0, 13), 1132939),
 ((0, 14), 1169192),
 ((0, 15), 1196336),
 ((0, 16), 1170027),
 ((0, 17), 1226865),
 ((0, 18), 1246305),
 ((0, 19), 1233307),
 ((0, 20), 1193495),
 ((0, 21), 1144991),
 ((0, 22), 1188082),
 ((0, 23), 1136498),
 ((0, 24), 1171027),
 ((0, 25), 1257640),
 ((0, 26), 1172161),
 ((0, 27), 1171267),
 ((0, 28), 1158241),
 ((0, 29), 1161176),
 ((0, 30), 1207314),
 ((0, 31), 1191679),
 ((0, 32), 1212365),
 ((0, 33), 1151200),
 ((0, 34), 1260592),
 ((0, 35), 1166960),
 ((0, 36), 1177580),
 ((0, 37), 1228129),
 ((0, 38), 1128280),
 ((0, 39), 1197121),
 ((0, 40), 1174699),
 ((0, 41), 1183543),
 ((0, 42), 1208107),
 ((0, 43), 1153855),
 ((0, 44), 1152943),
 ((0, 45), 1188817),
 ((0, 46), 1154121),
 ((0, 47), 1159208),
 (

將排序好的RDD重新用`map()`來把$((i,k),P_{ij})$映射到同一層以方便輸出

In [69]:
P_RDD_reshape = P_RDD_sort.map(lambda x : (x[0][0],x[0][1],x[1]) )

In [75]:
output = P_RDD_reshape.collect()

最後，輸出到檔案`output.txt`

In [76]:
f = open("output.txt", "w")
for i, j, v in output:
    f.write("{},{},{}\n".format(i, j, v))
# pprint(P_RDD_reshape)

In [ ]:
P_RDD_reshape.saveAsTextFile("hw1") 

## Finish

後來在檢查答案的時候發現一個小問題，如果用原本輸出到檔案的方式好像會漏掉最後的幾筆資料，不確定是不是因為overflow的關係  
但如果是用RDD的`saveAsTextFile()`去存的話就都會在（但就沒辦法變成原本題目的格式），所以看起來運算部分是沒問題但輸出這邊不知道該如何處理  
然後我的資料夾裡面有放`output.txt`以及用RDD輸出的`part-00000`和`part-00001`供助教檢查  

In [77]:
sc

<SparkContext master=local appName=hw1>